In [1]:
import tensorflow as tf

In [2]:
wordEmbeddingDimension = 70
vocabularySize=665000
labels=5000
filterSizes_paragraph = [3]
filterSizes_allPara=3
paragraphLength=100
num_filters_parargaph=15
num_filters_allPara=20
maxParagraph = 7

filterShapeOfAllPara =[filterSizes_allPara,3,1,num_filters_allPara]
fullyConnectedLayerInput = 2100


In [3]:
fullyConnectedLayerInput = 2100

In [4]:
import numpy as np
class DataParser:
    def __init__(self):
        self.data=[]
    
    def getDataFromfile(self,fname):
        self.counter =0
        self.totalPages=0
        f=open(fname)
        self.data=[]
        totalPages = int(f.readline())
#         print("total Pages",totalPages)
        count=0
        maxWordsInParagraph=paragraphLength
        maxParagraphs=maxParagraph
        totalLabels=labels

        dummyParagraph =[0]*maxWordsInParagraph

        # for pageId in f.readline():
        while True:
            pageId= f.readline()
            if not pageId :
                break

            labelCount=int(f.readline())
            labelsTemp=[0]*totalLabels
            for i in range(labelCount):
                tempLab=int(f.readline())
                if tempLab<totalLabels:
                    labelsTemp[tempLab]=1
            instancesCount=int(f.readline())
            instancesTemp=[]
            for i in range(instancesCount):
                tempInstance=f.readline().split()
                temp=[int(x) for x in tempInstance]
                for i in range(len(temp),maxWordsInParagraph):
                    temp.append(0)
                instancesTemp.append(temp[:maxWordsInParagraph])

            for i in range(instancesCount,maxParagraphs):
                instancesTemp.append(dummyParagraph)

            self.data.append((labelsTemp,instancesTemp[:maxParagraphs]))
#         self.data = data
        self.totalPages = len(self.data)
        f.close()
        
          
        
    def nextBatch(self):
        if self.counter >=self.totalPages:
            self.counter=0
        data= self.data[self.counter]
        self.counter+=1
        return data
    def restore(self):
        self.counter=0
training = DataParser()    
training.getDataFromfile("data/training.txt")

# print(training.nextBatch())
print(training.totalPages)
training.restore()
testing=  DataParser()
testing.getDataFromfile("data/testing.txt")
print(testing.totalPages)

2774
966


In [5]:
wordEmbedding = tf.Variable(tf.random_uniform([vocabularySize, wordEmbeddingDimension], -1.0, 1.0),name="wordEmbedding")

paragraphList = []
for i in range(maxParagraph):
    paragraphList.append(tf.placeholder(tf.int32,[paragraphLength],name="paragraphPlaceholder"+str(i)))

target = tf.placeholder(tf.float32,[labels],name="target")

In [6]:
def getParagraphEmbedding(paragraphWords):
    device_name='cpu'
    with tf.device(device_name): 
        paraEmbedding=tf.nn.embedding_lookup(wordEmbedding,paragraphWords)
    
    return tf.expand_dims(tf.expand_dims(paraEmbedding, -1),0)

In [7]:
# paragraphVector = getParagraphEmbedding(paragraphList[0])

In [8]:
# paragraphVector

In [9]:
def convLayeronParagraph(paragraphVector,filterSizes,num_input_channels,num_filters):
    
    pooled_outputs=[]
    for filter_size in filterSizes:
        shape = [filter_size,wordEmbeddingDimension,1,num_filters]
        
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.1),name="paragraphConvLayerW_"+str(filter_size))
        bias= tf.Variable(tf.constant(0.1, shape=[num_filters]),name="paragraphConvLayerB_"+str(filter_size))
        conv = tf.nn.conv2d(
                    paragraphVector,
                    weights,
                    strides=[1, 1, wordEmbeddingDimension, 1],
                    padding="SAME",
                    name="conv")
        
        h = tf.nn.relu(tf.nn.bias_add(conv, bias), name="relu")
        pool_length=5
        pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, pool_length, 1, 1],
                    strides=[1, pool_length, 1, 1],
                    padding='SAME',
                    name="pool")
        pooled_outputs.append(pooled)
    return tf.reshape(tf.concat(pooled_outputs,axis=0),[1,-1])
#     filter_size = 

In [10]:
# paragraphCNNOutput = convLayeronParagraph (paragraphVector,filterSizes,1,num_filters_parargaph)

In [11]:
# paragraphCNNOutput

In [12]:
def convLayerCombineParagraph(paragraphVectorList,filterSizes_paragraph,filterShapeOfAllPara,num_filters_parargaph,num_filters_allPara):
    
    paragraphCNNEmbedding=[]
    
    for paragraph in paragraphVectorList:
        paragraphVector = getParagraphEmbedding(paragraph)
        cnnEmbedding = convLayeronParagraph(paragraphVector,filterSizes_paragraph,1,num_filters_parargaph)
        paragraphCNNEmbedding.append(cnnEmbedding)
    
    allParagraph=tf.expand_dims(tf.expand_dims(tf.concat(paragraphCNNEmbedding,axis=0),-1),0)
    
    shape = filterShapeOfAllPara
    
    weights= tf.Variable(tf.truncated_normal(shape, stddev=0.1),name="paragraphConvLayer2W_"+str(filterShapeOfAllPara[0]))
    bias= tf.Variable(tf.constant(0.1, shape=[num_filters_allPara]),name="paragraphConvLayer2B_"+str(filterShapeOfAllPara[0]))
    
    conv = tf.nn.conv2d(
                    allParagraph,
                    weights,
                    strides=[1, 1, 1, 1],
                    padding="SAME",
                    name="conv")
    h = tf.nn.relu(tf.nn.bias_add(conv, bias), name="relu")
    return tf.reshape(allParagraph,[1,-1])
    

In [13]:

# print(convOutput)

In [14]:
# convOutput

In [15]:
def fullyConnectedLayer(convOutput,labels):
    shape = [fullyConnectedLayerInput,labels]
    weights =tf.Variable(tf.truncated_normal(shape, stddev=0.1),name="FC_W")
    bias = tf.Variable(tf.constant(0.1, shape=[labels]),name="FC_Bias")
    layer = tf.nn.sigmoid(tf.matmul(convOutput, weights) + bias)
    return layer

In [16]:
device_name='cpu'
with tf.device(device_name): 
    convOutput=convLayerCombineParagraph(paragraphList,filterSizes_paragraph,filterShapeOfAllPara,num_filters_parargaph,num_filters_allPara)
    prediction=fullyConnectedLayer(convOutput,labels)
    cross_entropy = -tf.reduce_sum(((target*tf.log(prediction + 1e-9)) + ((1-target) * tf.log(1 - prediction + 1e-9)) )  , name='xentropy' ) 
    cost = tf.reduce_mean(cross_entropy)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [17]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [19]:
epoch = 1
for e in range(epoch):
    for itr in range(training.totalPages):
        feed_dict_input={}
        data= training.nextBatch()
        feed_dict_input[target]=data[0]
        for p in range(maxParagraph):
            feed_dict_input[paragraphList[p]]= data[1][p]
        cross_e=session.run(optimizer,feed_dict=feed_dict_input)
#         cross_e=session.run(cross_entropy,feed_dict=feed_dict_input)
#         print(cross_e)
        if itr > 50:
            break
        

In [ ]:
target

In [20]:
saver = tf.train.Saver()

In [ ]:
saver.save(session, 'my_test_model')

In [ ]:
trueLabelsTest=[]
predLabelsTest=[]
for itr in range(testing.totalPages):
    feed_dict_input={}
    data= testing.nextBatch()
    feed_dict_input[target]=data[0]
    for p in range(maxParagraph):
        feed_dict_input[paragraphList[p]]= data[1][p]
    pred=session.run(prediction,feed_dict=feed_dict_input)
    
    trueLabelsTest.append(data[0])
    predLabelsTest.append(pred[0])
#     cross_e=session.run(cross_entropy,feed_dict=feed_dict_input)

In [ ]:
w=open("trueLabelesTest.txt","w")
for i in trueLabelsTest:
    w.write(str(i)+"\n")
w.close()

w=open("predLabelsTest.txt","w")
for j in predLabelsTest:
    for i in j:
        w.write(str(i)+" ")
    w.write("\n")
w.close()



In [ ]:
# cross_e=session.run(cross_entropy,feed_dict=feed_dict_input)
# print(cross_e)
# pred=session.run(prediction,feed_dict=feed_dict_input)
# print(pred)

In [ ]:
# for i in range(len(pred)):
#     if pred[0][i]>0.4:
#         print(i)
# pred

In [ ]:
# session = tf.Session()
# # session.run(tf.global_variables_initializer())
# new_saver = tf.train.import_meta_graph('my_test_model.meta')
# new_saver.restore(session, './my_test_model')


In [ ]:
# session.run(tf.global_variables_initializer())
# session.run(tf.all_variables())
# 

In [ ]:
print(session.run(prediction,feed_dict=feed_dict_input))
session.run(cross_entropy,feed_dict=feed_dict_input)

In [ ]:
# all_vars = tf.trainable_variables()
# for v in all_vars:
#     print(v)
#     print(session.run(v))

In [ ]:
# print([v.op.name for v in tf.all_variables()])

In [ ]:
ckpt = tf.train.get_checkpoint_state('.')

In [ ]:
# sess = tf.Session()

In [ ]:
# saver = tf.train.Saver()
# saver.restore(sess, ckpt.model_checkpoint_path)